In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
#cobra_model = load_json_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.json")



from cobra.io.mat import *
#from itertools import compress



cobra_model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")

import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [2]:
def get_data_for_heatmap(explanatory_subgraph_path):
        
    explanatory_subgraph = pd.read_parquet(explanatory_subgraph_path).reset_index(drop=True)

    ray.shutdown()
    ray.init()
    @ray.remote


    def get_genes(node_1, node_2):
        
        if node_2 in dict_rxn_to_gene.keys():
            
            return dict_rxn_to_gene[node_2]
            
        elif node_1 in dict_rxn_to_gene.keys():
            
            return dict_rxn_to_gene[node_1]
            
        else:
            return ['']
        
    g = [] 
    for node_1, node_2 in zip(explanatory_subgraph.node1.tolist(), explanatory_subgraph.node2.tolist()):   
        g.append(get_genes.remote(node_1, node_2 ))

    
    genes = pd.Series(ray.get(g) , name='genes')



    assert genes.shape[0] == explanatory_subgraph.shape[0]


    return pd.concat([genes, explanatory_subgraph], axis=1)


to_R_heatmap_explanatory_subgraph_masked_NONflux = get_data_for_heatmap("./results/dataframes/explanatory_subgraph_masked_NONflux.parquet.gzip")
to_R_heatmap_explanatory_subgraph_masked_FLUX = get_data_for_heatmap("./results/dataframes/explanatory_subgraph_masked_FLUX.parquet.gzip")

2022-12-05 16:47:30,468	INFO worker.py:1518 -- Started a local Ray instance.
2022-12-05 16:47:35,501	INFO worker.py:1518 -- Started a local Ray instance.


In [3]:
to_R_heatmap_explanatory_subgraph_masked_FLUX

,genes,node1,node2,0,1,2,3,4,5,6,...,490,491,492,493,494,495,496,497,498,499
0,[1384.1],CSNATr,acrn_c,0.012148,0.013416,0.011276,0.011888,0.096076,0.166618,0.011295,...,0.011613,0.184368,0.011355,0.011329,0.011254,0.016307,0.010661,0.011843,0.169054,0.049353
1,[8140.1],val_L_c,r1659,0.014589,0.031423,0.030059,0.013978,0.803187,0.790388,0.033168,...,0.021375,0.674615,0.038831,0.025976,0.020469,0.037111,0.023664,0.015999,0.792715,0.311713
2,[9057.1],val_L_c,r1758,0.019427,0.024003,0.023445,0.017814,0.663541,0.725609,0.024899,...,0.025530,0.446894,0.026860,0.021713,0.030820,0.460138,0.021926,0.019105,0.675851,0.360610
3,[9057.1],val_L_c,r1909,0.025714,0.185707,0.041876,0.021220,0.804908,0.761306,0.377098,...,0.402875,0.700829,0.710925,0.036056,0.903131,0.751229,0.078465,0.025082,0.797127,0.649089
4,[8140.1],val_L_c,r1554,0.011886,0.011325,0.011246,0.012258,0.095098,0.124077,0.011395,...,0.011012,0.091430,0.011054,0.011610,0.010834,0.074922,0.011403,0.012053,0.105984,0.084411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,[],asp_L_c,ASPLYSGLUr,0.015080,0.014520,0.016201,0.014632,0.046056,0.200517,0.014466,...,0.014014,0.037121,0.014496,0.015919,0.013781,0.107351,0.014789,0.015994,0.206875,0.041607
501,[],asp_L_c,TYRASPARGr,0.013919,0.013840,0.013825,0.013828,0.013604,0.212942,0.013892,...,0.013835,0.041500,0.013743,0.013781,0.013930,0.165055,0.013726,0.013870,0.270971,0.013575
502,[2805.1],asp_L_c,ASPTA,0.015542,0.014595,0.015240,0.014744,0.039506,0.121339,0.014347,...,0.014592,0.031746,0.015140,0.015525,0.014958,0.058102,0.015294,0.015355,0.184396,0.023219
503,"[10165.1, 8604.1]",asp_L_c,ASPGLUm,0.015708,0.014714,0.018214,0.014832,0.065192,0.228103,0.014527,...,0.015061,0.046203,0.016832,0.015949,0.015475,0.085914,0.015675,0.016054,0.374638,0.053085


In [4]:
to_R_heatmap_explanatory_subgraph_masked_NONflux

,genes,node1,node2,0,1,2,3,4,5,6,...,490,491,492,493,494,495,496,497,498,499
0,[8140.1],val_L_c,r1659,0.013759,0.663229,0.072295,0.794343,0.157651,0.047477,0.718332,...,0.192231,0.207602,0.063182,0.169742,0.013927,0.953045,0.013849,0.748983,0.320102,0.013883
1,[9057.1],val_L_c,r1909,0.013864,0.685872,0.075776,0.803910,0.169142,0.049186,0.735130,...,0.202378,0.225891,0.065112,0.179841,0.013858,0.954067,0.013894,0.764703,0.348414,0.013852
2,[8140.1],val_L_c,r1644,0.013881,0.650760,0.070430,0.790413,0.150546,0.046621,0.711348,...,0.186878,0.198251,0.062032,0.165233,0.013934,0.953053,0.013803,0.734956,0.311358,0.013822
3,[9057.1],val_L_c,r1818,0.013880,0.713899,0.082118,0.816506,0.191687,0.052090,0.760997,...,0.222079,0.261006,0.068843,0.195526,0.013936,0.955612,0.013781,0.791553,0.393312,0.013668
4,[9057.1],val_L_c,r1861,0.013794,0.686421,0.076701,0.806278,0.172001,0.049553,0.739204,...,0.206394,0.233031,0.065668,0.182333,0.013982,0.954286,0.013862,0.770544,0.357550,0.013841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,[83884.1],orn_c,r2404,0.013781,0.913564,0.030811,0.907537,0.906212,0.915990,0.896181,...,0.795431,0.904643,0.794685,0.786874,0.013979,0.037245,0.013769,0.034506,0.905911,0.013850
501,[83884.1],orn_c,r2405,0.013649,0.816948,0.023292,0.824100,0.023622,0.858237,0.699381,...,0.374301,0.029454,0.412794,0.374083,0.013877,0.032361,0.013920,0.034861,0.800937,0.013688
502,[83884.1],orn_c,r2412,0.013722,0.887993,0.029476,0.884063,0.870380,0.898668,0.851316,...,0.666100,0.858194,0.681195,0.657556,0.013824,0.032955,0.013842,0.034173,0.878155,0.013877
503,"[6519.1, 11136.1]",orn_c,ORNLEUrBATtc,0.013890,0.850336,0.014768,0.851277,0.804333,0.875825,0.774388,...,0.489275,0.754971,0.516924,0.486308,0.013869,0.048107,0.013824,0.204582,0.842319,0.013805


In [6]:
to_R_heatmap_explanatory_subgraph_masked_FLUX.to_csv(
    "./results/dataframes/to_R_heatmap_explanatory_subgraph_masked_FLUX.csv")
to_R_heatmap_explanatory_subgraph_masked_NONflux.to_csv(
    "./results/dataframes/to_R_heatmap_explanatory_subgraph_masked_NONflux.csv")

# Masked

In [4]:
Masked_edges_node_names_explainer_subgraphs = pd.read_parquet(
    "./results/dataframes/Masked_edges_node_names_explainer_subgraphs.parquet.gzip").reset_index(drop=True)

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(Masked_edges_node_names_explainer_subgraphs.node1.tolist(), Masked_edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == Masked_edges_node_names_explainer_subgraphs.shape[0]



Masked_results_for_Heatmaps = pd.concat([genes, Masked_edges_node_names_explainer_subgraphs], axis=1)
Masked_results_for_Heatmaps

2022-12-05 12:25:49,703	INFO worker.py:1518 -- Started a local Ray instance.


,genes,node1,node2,0,1
0,[],TRPGLULEUr,glu_L_c,0.014967,0.014388
1,"[1375.1, 1375.3, 126129.1, 1374.1, 1375.2]",C160CPT1,coa_c,0.015270,0.022956
2,"[1375.1, 1375.3, 126129.1, 1374.1, 1375.2]",C160CPT1,crn_c,0.014483,0.015676
3,[51727.1],CYTK10,cdp_c,0.013181,0.016316
4,[9057.1],r1758,trp_L_c,0.012910,0.016337
...,...,...,...,...,...
500,[160728.1],L_LACt4r,lac_L_e,0.012621,0.017381
501,[51727.1],CYTK5,dcmp_c,0.012556,0.017218
502,[],AHCYStr,ahcys_r,0.012489,0.017030
503,"[6530.1, 6531.1]",DOPAt4_2_r,dopa_c,0.012889,0.016586


In [5]:
Masked_results_for_Heatmaps.to_csv(
    "./results/dataframes/Masked_results_for_Heatmaps.csv")